In [329]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve

API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [464]:
df_holdet = get_holdet_data()
# df_winner = get_tournament_winner_odds(API_KEY)
# df_group_stage = get_group_stage_odds(API_KEY)
# df_get_group_stage_OU = get_group_stage_OU(API_KEY)
# df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
# df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [193]:
import scipy.optimize as opt
import math

def func(zGuess, _lambda):
    val = _lambda
    x = zGuess
    eq_1 = math.exp(-x)+math.exp(-x)*x+(math.exp(-x)*x**2)/2-val 
    return eq_1[0]

zGuess = 2.4
df_get_group_stage_OU['lambda']= df_get_group_stage_OU['prob_under'].apply(lambda x: opt.fsolve(func,zGuess,args=(x))[0])
df_get_group_stage_OU['xG'] = df_get_group_stage_OU['lambda']

In [194]:
df_group_stage = df_group_stage.merge(df_get_group_stage_OU[['home_team','away_team','xG']], on = ['home_team','away_team'])
df_group_stage["xGH"] = df_group_stage["xG"]*df_group_stage["prob1"]/(df_group_stage["prob1"]+df_group_stage["prob2"])
df_group_stage["xGA"] = df_group_stage["xG"]*df_group_stage["prob2"]/(df_group_stage["prob1"]+df_group_stage["prob2"])

In [195]:
df_schedule = pd.read_csv("common/ressources/schedule.csv")
df_schedule.rename(columns = {"Home Team":"home_team", "Away Team":"away_team", "Round Number":"round"}, inplace = True)
df_schedule.loc[(df_schedule["home_team"]=="Korea Republic"),"home_team"]="South_Korea"
df_schedule.loc[(df_schedule["away_team"]=="Korea Republic"),"away_team"]="South_Korea"
df_group_stage.loc[(df_group_stage["home_team"]=="South Korea"),"home_team"]="South_Korea"
df_group_stage.loc[(df_group_stage["away_team"]=="South Korea"),"away_team"]="South_Korea"
df_group_stage = df_group_stage.merge(df_schedule[["home_team","away_team","round"]], on = ['home_team','away_team'])
df_group_stage.sort_values("round", inplace=True)
df_group_stage.reset_index(drop=True, inplace = True)

In [465]:
df_holdet["selected"]=False
for team in df_holdet["country"].unique():
    df_temp = df_holdet[df_holdet["country"]==team]
    df_temp.sort_values("price", ascending = False)
    df_holdet.loc[df_holdet["name1"].isin(df_temp.reset_index().loc[0:11,"name1"]),"selected"]=True

In [466]:
df_holdet["G"] = np.random.uniform(0,1,len(df_holdet))
df_holdet["A"] = np.random.uniform(0,1,len(df_holdet))
df_holdet["Yellow"] = np.random.uniform(0,1,len(df_holdet))
df_holdet["Red"] = np.random.uniform(0,1,len(df_holdet))

In [204]:
# __df_group_stage = df_group_stage.copy()
df_group_stage = __df_group_stage.copy()

In [205]:
df_group_stage["home_team"] = df_group_stage["home_team"].apply(lambda x: eng2dan[x])
df_group_stage["away_team"] = df_group_stage["away_team"].apply(lambda x: eng2dan[x])

In [218]:
# _df_holdet = df_holdet.copy()
# _df_group_stage = df_group_stage.copy()
df_holdet = _df_holdet.copy()
df_group_stage = _df_group_stage.copy()
len(df_holdet)

845

In [467]:
for round in ["1","2","3"]:
    df_temp1 = df_holdet[["name1","name2","country"]].merge(df_group_stage.loc[df_group_stage["round"]==round,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
    df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

    df_temp2 = df_holdet[["name1","name2","country"]].merge(df_group_stage.loc[df_group_stage["round"]==round,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
    df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

    df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
    df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [455]:
df_winner["name"] = df_winner["name"].replace("South Korea","South_Korea")
df_winner["country"] = df_winner["name"].apply(lambda x: eng2dan[x])

In [456]:
temp_list = []
for team in df_group_stage["home_team"].unique():
    xG, pwin, xGa = 0, 0, 0
    xG += df_group_stage.loc[df_group_stage["home_team"]==team,"xGH"].values[0]
    xGa += df_group_stage.loc[df_group_stage["home_team"]==team,"xGA"].values[0]
    pwin += (df_group_stage.loc[df_group_stage["home_team"]==team,"prob1"]/(df_group_stage.loc[df_group_stage["home_team"]==team,["prob1","prob2"]].sum(axis=1))).sum()
    xG += df_group_stage.loc[df_group_stage["away_team"]==team,"xGA"].values[0]
    xGa += df_group_stage.loc[df_group_stage["away_team"]==team,"xGH"].values[0]
    pwin += (df_group_stage.loc[df_group_stage["away_team"]==team,"prob2"]/(df_group_stage.loc[df_group_stage["away_team"]==team,["prob1","prob2"]].sum(axis=1))).sum()
    temp_list.append([team, xG/3, pwin/3, xGa/3])
    
df_temp = pd.DataFrame(temp_list)
df_temp.columns = ["country","xG","pwin", "xGa"]
df_winner = df_winner.merge(df_temp, on = "country")

In [457]:
sims = joblib.load("common/ressources/sims")
df_winner["name"] = df_winner["name"].replace("South_Korea","South Korea")
for country in df_winner["name"]:
    R16, QF, SF, Final = get_play_off_opps(country, sims)
    df_winner.loc[(df_winner["name"]==country),"R16"] = R16
    df_winner.loc[(df_winner["name"]==country),"QF"] = QF
    df_winner.loc[(df_winner["name"]==country),"SF"] = SF
    df_winner.loc[(df_winner["name"]==country),"Final"] = Final
    
df_grid = pd.read_csv("common/ressources/df_grid")
df_grid.columns = ["name", "R16_prob", "QF_prob", "SF_prob" ,"F_prob","W_prob"]
df_winner = df_winner.merge(df_grid, on = "name")

In [458]:
df_winner["Final"] = df_winner["Final"].replace("null","Argentina")
for country in df_winner["name"]:
    df_winner.loc[df_winner["name"]==country,"prob_to_win_R16"] = df_winner.loc[df_winner["name"] == df_winner.loc[df_winner["name"]==country,"R16"].values[0],"prob_to_win"].values[0]
    df_winner.loc[df_winner["name"]==country,"prob_to_win_QF"] = df_winner.loc[df_winner["name"] == df_winner.loc[df_winner["name"]==country,"QF"].values[0],"prob_to_win"].values[0]
    df_winner.loc[df_winner["name"]==country,"prob_to_win_SF"] = df_winner.loc[df_winner["name"] == df_winner.loc[df_winner["name"]==country,"SF"].values[0],"prob_to_win"].values[0]
    df_winner.loc[df_winner["name"]==country,"prob_to_win_Final"] = df_winner.loc[df_winner["name"] == df_winner.loc[df_winner["name"]==country,"Final"].values[0],"prob_to_win"].values[0]

df_winner["prob_to_win_R16"] = df_winner["prob_to_win"]/(df_winner["prob_to_win_R16"]+df_winner["prob_to_win"])
df_winner["prob_to_win_QF"] = df_winner["prob_to_win"]/(df_winner["prob_to_win_QF"]+df_winner["prob_to_win"])
df_winner["prob_to_win_SF"] = df_winner["prob_to_win"]/(df_winner["prob_to_win_SF"]+df_winner["prob_to_win"])
df_winner["prob_to_win_Final"] = df_winner["prob_to_win"]/(df_winner["prob_to_win_Final"]+df_winner["prob_to_win"])

df_winner["xG_R16"] = df_winner["prob_to_win_R16"]/df_winner["pwin"]*df_winner["xG"]
df_winner["xG_QF"] = df_winner["prob_to_win_QF"]/df_winner["pwin"]*df_winner["xG"]
df_winner["xG_SF"] = df_winner["prob_to_win_SF"]/df_winner["pwin"]*df_winner["xG"]
df_winner["xG_Final"] = df_winner["prob_to_win_Final"]/df_winner["pwin"]*df_winner["xG"]

df_winner["xGa_R16"] = df_winner["prob_to_win_R16"]/df_winner["pwin"]*df_winner["xGa"]
df_winner["xGa_QF"] = df_winner["prob_to_win_QF"]/df_winner["pwin"]*df_winner["xGa"]
df_winner["xGa_SF"] = df_winner["prob_to_win_SF"]/df_winner["pwin"]*df_winner["xGa"]
df_winner["xGa_Final"] = df_winner["prob_to_win_Final"]/df_winner["pwin"]*df_winner["xGa"]
df_winner.to_csv("common/ressources/df_winner", index = False)

In [474]:
df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'F_prob', 'W_prob',
                                       'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', 
                                       'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

In [520]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G']/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G']/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"Exptectred_points"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 7000
        # group games
        for i in range(1,4):
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'Yellow'].values[0] * -20000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'Yellow'].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'Yellow'].values[0]*-20000**(1/i)
            points += df_holdet.loc[df_holdet["name1"]==player,f'Red'].values[0] * -20000**(1/i)
            if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
                points += np.exp(df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000**(1/i)
                
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
                points += np.exp(df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000**(1/i)
                points += df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000**(1/i)
                points += df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/i)
            
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
                points += df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000**(1/i)
                points += df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/i)
            
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
                points += df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000**(1/i)
                points += df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/i)
        
        df_holdet.loc[df_holdet["name1"]==player,"Exptectred_points"] = points

In [524]:
df_holdet.to_csv("df_holdet",index=False)